<a href="https://colab.research.google.com/github/baswenneker/next-in-ai/blob/master/next_in_ai_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarize Pocket articles and save to docx.
For more info, go to the [GitHub](https://github.com/baswenneker/next-in-ai) repo.

In [6]:
#@title Step 1: Configuration. Add keys and urls.

# Get it here: https://help.getpocket.com/article/1074-can-i-subscribe-to-my-list-via-rss
os.environ['POCKET_RSS_FEED'] = "https://getpocket.com/users/*sso1456990609615e33/feed/all"
if os.environ['POCKET_RSS_FEED'] == "https://getpocket.com/users/<YOURPOCKETUSERNAME>/feed/all":
  raise Exception("❌ You forgot to add the url to your Pocket RSS feed.")

# Enter your API key (find it here https://platform.openai.com/account/api-keys).
os.environ['OPENAI_API_KEY'] = getpass('Enter your paid OpenAI API key here (find it here https://platform.openai.com/account/api-keys):')

# Beehiiv newsletter page (used to determine which Pocket articles are fetched).
# Leave empty to get all articles from the last 7 days.
# For example: "https://nextinai.beehiiv.com/"
os.environ['BEEHIIV_URL']=''

# Use "gpt-3.5-turbo" (default) or "gpt-4". GPT-4 gives the best results, but is more expensive.
os.environ['MODEL'] = "gpt-3.5-turbo"

# Leave empty for English:
os.environ['OUTPUT_LANGUAGE'] ="Dutch" 

Enter your paid OpenAI API key here (find it here https://platform.openai.com/account/api-keys):··········


In [7]:
#@title Step 2: Set up and install requirements.

%load_ext autoreload
%autoreload
from getpass import getpass
import os

!rm -rf /content/next-in-ai/
!git clone https://github.com/baswenneker/next-in-ai.git

!pip install -r /content/next-in-ai/.devcontainer/requirements.txt

import sys
sys.path.insert(1, "/content/next-in-ai")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Cloning into 'next-in-ai'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 107 (delta 59), reused 86 (delta 40), pack-reused 0
Receiving objects: 100% (107/107), 81.09 KiB | 1.29 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now we're ready to fetch the urls from Pocket.

In [10]:
#@title Step 3: Fetch new Pocket article urls.

%autoreload
from next_in_ai.PocketParser import PocketParser

p = PocketParser()

# Get all Pocket urls since you sent out the last newsletter (works with Beehiiv):
# latest_pocket_urls = p.new_articles()
# Get all Pocket urls added since 7 days:
latest_pocket_urls = p.new_articles_from_days_ago(2)

print("We got the following URLs from Pocket:")

for url in latest_pocket_urls:
  print(f" - {url}")

We got the following URLs from Pocket:
 - https://huggingface.co/blog/starcoder?utm_source=pocket_saves
 - https://semianalysis.com/p/google-we-have-no-moat-and-neither?r=8ryw
 - https://kdnuggets.com/2023/05/hugginggpt-secret-weapon-solve-complex-ai-tasks.html


Now, try to fetch all the content from the urls and summarize it with GPT.

⚠️ Disclaimer: some websites might be unavailable or block our crawler on purpose.

In [11]:
#@title Step 4: Scrape the articles and summarize using OpenAI.

%autoreload
from next_in_ai.BatchSummarizer import BatchSummarizer

batch_summarizer = BatchSummarizer(latest_pocket_urls)
batch_summarizer.create_summary_document(debug=False)

ERROR:trafilatura.downloads:retries/redirects: https://semianalysis.com/p/google-we-have-no-moat-and-neither?r=8ryw HTTPSConnectionPool(host='semianalysis.com', port=443): Max retries exceeded with url: /p/google-we-have-no-moat-and-neither?r=8ryw (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f42366c3f40>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
ERROR:trafilatura.utils:lxml parsing failed: Document is empty
ERROR:trafilatura.core:empty HTML tree for URL None


📄 Creating the summary document


Summarizing article:  https://huggingface.co/blog/starcoder?utm_source=pocket_saves
🤗 Got summary from cache.


Summarizing article:  https://semianalysis.com/p/google-we-have-no-moat-and-neither?r=8ryw
🌎 Getting the contents of the url.
<next_in_ai.OpenAISummarizer.OpenAISummarizer object at 0x7f42366c3760> None
❌ Couldn't fetch content from https://semianalysis.com/p/google-we-have-no-moat-and-neither?r=8ryw


Summarizing article:  https://kdnuggets.com/2023/05/hugginggpt-secret-weapon-solve-complex-ai-tasks.html
🌎 Getting the contents of the url.
<next_in_ai.OpenAISummarizer.OpenAISummarizer object at 0x7f42366c37f0> HuggingGPT: The Secret Weapon to Solve Complex AI Tasks
Get ready to discover the next big thing in AI with HuggingGPT. Read this article to develop an understanding of how it works and how it handles complex AI tasks.
Image by Author
Introduction
Have you heard of the term Artificial General Intelligence (AGI)? If not, let me clarify. 

In [12]:
#@title Step 5: Download the resulting .docx. Finished! 
from google.colab import files
import datetime

date_str = datetime.datetime.now().strftime("%Y-%m-%d")
output_filename = f"summaries-{date_str}.docx"
files.download(f"/content/{output_filename}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>